In [2]:
import numpy as np
import pandas as pd
import csv

import datetime
from datetime import datetime

# crime report

In [3]:
report = pd.read_csv("/project/graziul/data/CPD/crime/Crimes_-_2001_to_Present.csv")
report = report[['Date','Primary Type','Beat']]

# extract date and time
report['Date'] = pd.to_datetime(report['Date'], format='%m/%d/%Y %I:%M:%S %p')

# get date
report['day'] = report['Date'].astype(str).str[:10]

# get target years
report = report.loc[(report["day"]>="2018-07-01") & (report["day"]<="2020-07-01")]

In [29]:
report.head()

,Date,Primary Type,Beat,day,Violent,Property,Total
2,2018-09-01 00:01:00,THEFT,631,2018-09-01,False,True,True
90,2020-03-17 21:30:00,THEFT,1634,2020-03-17,False,True,True
183,2019-09-24 08:00:00,DECEPTIVE PRACTICE,132,2019-09-24,False,False,True
235,2019-10-13 20:30:00,THEFT,1221,2019-10-13,False,True,True
278,2018-12-20 16:00:00,OTHER OFFENSE,1724,2018-12-20,False,False,True


In [28]:
# label violent and property crime
violent_crime = ["HOMICIDE","ASSAULT","BATTERY"]
property_crime = ["BURGLARY","THEFT","MOTOR VEHICLE THEFT"]

report['Violent'] = np.where(report['Primary Type'].isin(violent_crime),True,False)
report['Property'] = np.where(report['Primary Type'].isin(property_crime),True,False)

report['Total'] = True

In [133]:
# groupby summary (hour)
#report = report[["day","Zone","Hour","Violent","Property","Arrest","Domestic","Total","Primary Type"]]
#report = report.sort_values(by=['day','Zone',"Hour"])
#report_sum = report.groupby(["day","Hour","Zone"]).sum()

In [30]:
# groupby summary (day)
report = report[["day","Beat","Violent","Property","Total","Primary Type"]]
report = report.sort_values(by=['day','Beat'])
report_sum = report.groupby(["day","Beat"]).sum()

In [66]:
#report_sum.reset_index().to_csv('/home/xuranzeng/project/data/crimes_data_beat_simplified_summary_day.csv')


In [64]:
pd.DataFrame(report_sum).reset_index()

,day,Beat,Violent,Property,Total
0,2018-07-01,111,0,4,4
1,2018-07-01,112,1,3,4
2,2018-07-01,113,2,2,4
3,2018-07-01,114,1,1,2
4,2018-07-01,121,0,3,6
...,...,...,...,...,...
172127,2020-07-01,2525,0,0,2
172128,2020-07-01,2531,0,0,2
172129,2020-07-01,2532,2,0,3
172130,2020-07-01,2533,1,1,2


# 311 requests

In [35]:
request = pd.read_csv("/project/graziul/ra/xuranzeng/request_simplified_type.csv")[['CREATED_DATE','POLICE_BEAT','TYPE2']]

# extract date and time
request['Date'] = pd.to_datetime(request['CREATED_DATE'], format='%m/%d/%Y %I:%M:%S %p')

# get date
request['day'] = request['Date'].astype(str).str[:10]

# get target years
request = request.loc[(request["day"]>="2018-07-01") & (request["day"]<="2020-07-01")]



/software/python-anaconda-2020.11-el8-x86_64/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
# label public and private request

request['public'] = np.where(request['TYPE2']=='public',True,False)
request['private'] = np.where(request['TYPE2']=='private',True,False)
request['other'] = np.where(request['TYPE2']=='other',True,False)

request = request[["day","POLICE_BEAT","public","private","other"]]

In [46]:
request.head()

,day,POLICE_BEAT,TYPE2,public,private,other
480201,2018-07-01,422.0,public,True,False,False
480199,2018-07-01,611.0,public,True,False,False
480182,2018-07-01,624.0,public,True,False,False
480203,2018-07-01,713.0,public,True,False,False
480198,2018-07-01,811.0,public,True,False,False


In [48]:
# groupby summary (day)

request = request.sort_values(by=['day','POLICE_BEAT'])
request_sum = request.groupby(["day","POLICE_BEAT"]).sum()

In [58]:
request_sum.rename(columns = {'POLICE_BEAT':'Beat'}, inplace = True)

In [57]:
request_sum

public  private  other
day        POLICE_BEAT                        
2018-07-01 422.0             1        0      0
           611.0             1        0      0
           624.0             1        0      0
           713.0             1        0      0
           811.0             1        0      0
...                        ...      ...    ...
2020-07-01 2531.0            3        2      0
           2532.0            9        5      0
           2533.0           11        4      0
           2534.0           12        6      0
           2535.0           26        2      0

[149116 rows x 3 columns]

In [67]:
#request_sum.reset_index().to_csv('/home/xuranzeng/project/data/request_beat_simplified_summary_day.csv')


# Align report & request

In [87]:
report1 = report_sum.reset_index()
request1 = request_sum.reset_index()

request1.columns = ['day', 'Beat', 'public', 'private','other']

request1['Beat'] = request1['Beat'].astype(int).astype(str)
report1['Beat'] = report1['Beat'].astype(int).astype(str)

In [88]:
report1.head()

,day,Beat,Violent,Property,Total
0,2018-07-01,111,0,4,4
1,2018-07-01,112,1,3,4
2,2018-07-01,113,2,2,4
3,2018-07-01,114,1,1,2
4,2018-07-01,121,0,3,6


In [89]:
request1.head()

,day,Beat,public,private,other
0,2018-07-01,422,1,0,0
1,2018-07-01,611,1,0,0
2,2018-07-01,624,1,0,0
3,2018-07-01,713,1,0,0
4,2018-07-01,811,1,0,0


In [91]:
report_request_beat = pd.merge(report1, request1, on=['day', 'Beat'])

In [92]:
report_request_beat.head()

,day,Beat,Violent,Property,Total,public,private,other
0,2018-07-01,422,1,0,3,1,0,0
1,2018-07-01,611,3,0,6,1,0,0
2,2018-07-01,624,0,2,9,1,0,0
3,2018-07-01,713,1,2,4,1,0,0
4,2018-07-01,811,0,0,3,1,0,0


In [93]:
#report_request_beat.to_csv('/home/xuranzeng/project/data/report_request_beat.csv')
